# Importing ORACC Data from corpus.json
by Niek Veldhuis
UC Berkeley

February 2017

# TODO
* check that COFs are treated properly
* check that lines that continue into the next line (as in bilinguals) are captured completely.


# Introduction

Purpose of the code is to download [ORACC](http://oracc.org) JSON files that contain textual data and produce a `.csv` file with the relevant data for use in computational text analysis. This comes in the place of scraping the published `html` (see the [Scrape-ORACC](https://github.com/niekveldhuis/Digital-Assyriology/tree/master/Scrape-Oracc) repo). The JSON files contain all the transliteration and lemmatization data of an ORACC project (metadata are made available in a separate `.json` file). For an introduction to the various ORACC JSON files see the [ORACC Open Data](http://oracc.org/doc/opendata) page.

The resulting data file may include various elements of the ORACC data structure. The current code will output a file with the following fields: 

* id_text 
* lemma (listing all the lemmas of that text in sequence)

The selection and arrangement of fields may be adjusted with standard `Pandas` functions.

## Notes
The current version of the script works with the `ijson` library. Documentation for [ijson](https://www.dataquest.io/blog/python-json-tutorial/), unfortunately, is extremely brief. The notebook has been tested, using a list of 106 P, Q, X numbers (in `ob_lists_wood.txt`). Downloading the `.json` files from the ORACC server takes between 30 and 45 seconds. The rest of the script is reasonably fast. With larger lists of text IDs the script will obviously take longer. 

This notebook is written for **Python 3.5** with **Pandas 0.19** and **ijson 2.3**.

The notebook was originally written for the [Phylogenetics](https://github.com/ErinBecker/digital-humanities-phylogenetics) project with Erin Becker of [Data Carpentry](http://www.datacarpentry.org). The particular data selection and data manipulation performed in this notebook are adapted for use in the word2vec project. It should be fairly easy to adapt the notebook to the purposes of any other project that wishes to use [ORACC](http://oracc.org) data.

## Licensing
This notebook may be downloaded, used and adapted without any restrictions.

In [1]:
import pandas as pd   
import ijson
import urllib.request
import re
import tqdm
import numpy as np

# Input List of Text IDs or a project abbreviation
Identify a list of text IDs (P, Q, and X numbers) in the directory `input`. The IDs are six-digit P, Q, or X numbers preceded by a project abbreviation in the format 'PROJECT/P######' or 'PROJECT/SUBPROJECT/Q######'. For example:
* dcclt/P117395
* etcsri/Q001203
* rinap/rinap1/Q003421

The list should be created with a flat text editor such as Textedit or Emacs, and the filename should end in `.txt`.

Alternatively, one may enter the name (abbreviation) of a project or sub-project in [ORACC](http://oracc.org) and pull all the lemmatized data from that project. Note that the script will not automatically pull data from subprojects, they have to be requested separately. Examples:
* saao/saa01
* aemw/amarna
* rimanum

In [2]:
name = input('Filename or project abbreviation: ')

Filename or project abbreviation: hbtin


In [3]:
if name[-4:] == '.txt':
    textids = 'text_ids/' + name
    with open(textids, 'r') as f:
        pqxnos = f.readlines()
    pqxnos = [x.strip() for x in pqxnos]
    files = [x[:-7] + 'corpusjson/' + x[-7:] + '.json' for x in pqxnos]
    pqxnos = [x[-7:] for x in pqxnos]
    pqxnos[:5], files[:5]
else:
    files = []
    json = '/metadata.json'
    url = 'http://oracc.museum.upenn.edu/' + name + json
    try:
        d = urllib.request.urlopen(url)
        parser = ijson.parse(d)
        for prefix, event, value in parser:
            #print(prefix)
            if prefix == 'formats.lem.item':   # get only lemmatized texts
                files.append(name + '/corpusjson/' + value + '.json')
    except:
        print(url + ' not available')

# Parse
The function `oraccjsonparser()` takes one argument (the **url** of the `.json` file). It looks for the prefix `textid` to retrieve the six-digit P, Q, or X number of the text artifact. Parsing the file sequentially the code looks for the places where a line starts (`'.type' = 'line-start'`) and where a word starts (`'.node' = 'l'`, where `l` is for "lemma"). At each level the code will retrieve the relevant data and create a list where each entry is a dictionary that represents a single word. 

Words not only include lemmatized words, but also unlemmatized and unlemmatizable words (such as breaks).

The dictionary includes the keys `id_line` and `id_word` that allow the user to reassemble words and lines in order.

In [4]:
def oraccjsonparser(url):
    d = urllib.request.urlopen(url)
    parser = ijson.parse(d)
    word_l = []
    word_d = {}
    line_start = False
    word_start = False
    nonx = False
    for prefix, event, value in parser:
        if prefix == 'textid':
            id_text = value
#            print("parsing " + value)
        if prefix.endswith('.type'):
            if value == 'line-start':
                line_start = True
            else:
                line_start = False
        if line_start:
            if prefix.endswith('.ref') and not word_start:
                id_line = value # id_line is a reference number for a line
                                # that includes the id_text (e.g. P123456.49)
            if prefix.endswith('.label'):
                label = value   # label is a human-readable line number of the format
                                # o ii 24' (obverse column 2 line 24')
        if prefix.endswith('node'):
            if value == 'l':
                #print(prefix)
                word_start = True
                if not word_d == {}:
                    word_l.append(word_d) # append the previous word to the list
                word_d = {}               # and start a new dictionary
                #word_d['id_text'] = id_text # provide each word with appropriate 
                #word_d['id_line'] = id_line # text and line-ID
                word_d['label'] = label     # and the line label.
            else:
                word_start = False
        if word_start:
            if prefix.endswith('cdl.item.ref') or prefix.endswith('choices.item.ref'):
                #print(prefix), print(value)
                word_d['id_word'] = value
                id_elements = value.split('.')
                word_d['id_text'] = id_elements[0]
                word_d['id_line'] = id_elements[0] + '.' + id_elements[1]
            if prefix.endswith('.sig'):
                word_d['signature'] = value
            if '.f.' in prefix:
                category = re.sub('.*\.', '', prefix) # get element after the last dot of the prefix
                word_d[category] = value # copy each element into the dictionary
        if prefix.endswith('.type'):
            if value == 'nonx':
                nonx = True
            else:
                nonx = False
        if nonx:                         # this captures so-called $-lines with information
            if prefix.endswith('.ref'):  # about number of broken lines/columns.
                id_line = value          # $-lines have their own id_line.
            if prefix.endswith('.strict'):
                if value == '1':           # select only 'strict' $ lines
                    if not word_d == {}:
                        word_l.append(word_d)
                    word_d = {}
                    word_d['id_line'] = id_line
                    word_d['id_text'] = id_text
                else:
                    nonx = False
            if prefix.endswith('.extent'): # capture the three elements of strict $ lines
                word_d['extent'] = value   # namely extent, scope, and state.
            if prefix.endswith('.scope'):
                word_d['scope'] = value
            if prefix.endswith('.state'):
                word_d['state'] = value

    word_l.append(word_d)  # make sure that the last word is captured, too.
    return(word_l) # return a list of dictionaries, where each entry (dictionary) in
                   # the list represents a word.

# Call the Parser Function for Each Textid

In [5]:
url_prefix = "http://oracc.museum.upenn.edu/"
word_l = []
for id_text in tqdm.tqdm(files):   # tqdm creates a progress bar
    try:
        url = url_prefix + id_text
        word_l.extend(oraccjsonparser(url))
    except:
        print(url + ' not available') 

100%|██████████| 451/451 [06:55<00:00,  1.05s/it]


# Transform the Data into a DataFrame

In [6]:
words = pd.DataFrame(word_l)
words.head()
#words.loc[words['id_text'] == 'P357082']

,cf,epos,extent,form,gw,id_line,id_text,id_word,label,lang,norm,pos,scope,sense,signature,state
0,Anu-ah-ittannu,PN,NaN,{m}{d}60-ŠEŠ-MU{+nu},00,P235192.3,P235192,P235192.3.1,o 1,akk-x-ltebab,Anu-ah-ittannu,PN,NaN,00,@hbtin%akk-x-ltebab:{m}{d}60-ŠEŠ-MU{+nu}=Anu-a...,NaN
1,māru,N,NaN,DUMU,son,P235192.3,P235192,P235192.3.2,o 1,akk-x-ltebab,māru,N,NaN,son,@hbtin%akk-x-ltebab:DUMU=māru[son//son]N'N$māru,NaN
2,ša,DET,NaN,ša₂,of,P235192.3,P235192,P235192.3.3,o 1,akk-x-ltebab,ša,DET,NaN,of,@hbtin%akk-x-ltebab:ša₂=ša[of//of]DET'DET$ša,NaN
3,Nidintu-Anu,PN,NaN,{m}ni-din-tu₄-{d}60,00,P235192.3,P235192,P235192.3.4,o 1,akk-x-ltebab,Nidintu-Anu,PN,NaN,00,@hbtin%akk-x-ltebab:{m}ni-din-tu₄-{d}60=Nidint...,NaN
4,māru,N,NaN,DUMU,son,P235192.3,P235192,P235192.3.5,o 1,akk-x-ltebab,māri,N,NaN,son,@hbtin%akk-x-ltebab:DUMU=māru[son//son]N'N$māri,NaN


# Remove Spaces and Commas from Guide Word and Sence
Spaces in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens or nothing, respectively.

In [7]:
words = words.fillna('') # first replace Missing Values by empty string
words['sense'] = [x.replace(' ', '-') for x in words['sense']]
words['sense'] = [x.replace(',', '') for x in words['sense']]
words['gw'] = [x.replace(' ', '-') for x in words['gw']]
words['gw'] = [x.replace(',', '') for x in words['gw']]

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* gw (Guide Word: main or first translation in standard dictionary)
* id_line (a line ID that begins with the six-digit P, Q, or X number of the text)
* id_text (six-digit P, Q, or X number)
* id_word (word ID that begins with the ID number of the line)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* signature (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `pos`, `id_word`, `id_line`, and `id_text` should always be there. An unlemmatized word has `pos` 'X' (for unknown). Broken words have `pos` 'u' (for 'unlemmatizable).

# Manipulate
The columns may be manipulated with standard Pandas methods to create the desired output. By way of example, the following code will create a column `lemma` with the format **cf[gw]pos** (for instance **lugal[king]N**). For words that have no lemmatization `lemma` equals `form`. Only Sumerian words are allowed (and thus `lang` can be omitted) and in addition to the column `lemma` the column `base` is preserved; words that have no lemmatization take `form` as their base. Words and bases are concatenated to lines.

## Remove  non-Akkadian words

In [8]:
lang = ['akk', ''] # note that 'lang' is empty in entries that indicate damage
words = words.loc[words['lang'].str[:3].isin(lang)].reset_index()

## Create Lemma Column and Adjust Base

In [9]:
words['lemma'] = words['cf'] # first element of lemma is the citation form
words['lemma'] = [words['lemma'][i] + '[' + words['gw'][i] + ']' + words['pos'][i] 
                  if not words['lemma'][i] == '' 
                  else words['form'][i] +'[NA]NA' for i in range(len(words))]
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ]
#words['base'] = [words['base'][i] if not words['base'][i] == '' 
#                 or words['label'][i] == '' else words['form'][i] 
#                 for i in range(len(words))]
lemmas = words[['lemma', 'id_text', 'id_line', 'id_word', 'label']]#, 'extent', 'scope']]
#lemmas = words[['lemma', 'base', 'id_text', 'id_line', 'id_word', 'label', 'extent', 'scope']]
lemmas.head()

,lemma,id_text,id_line,id_word,label
0,Anu-ah-ittannu[00]PN,P235192,P235192.3,P235192.3.1,o 1
1,māru[son]N,P235192,P235192.3,P235192.3.2,o 1
2,ša[of]DET,P235192,P235192.3,P235192.3.3,o 1
3,Nidintu-Anu[00]PN,P235192,P235192.3,P235192.3.4,o 1
4,māru[son]N,P235192,P235192.3,P235192.3.5,o 1


## Group by Line

In [12]:
lines = words.groupby([words['id_line'], words['label']]).agg({
        'lemma': ' '.join,
        #'base': ' '.join,
        #'extent': ''.join, 
        #'scope': ''.join
    }).reset_index()
        

In [13]:
df = lines[['id_line', 'label', 'lemma']] #, 'base', 'extent', 'scope']]

In [14]:
df['id_text'] = df['id_line'].str[:7] # id_text was lost in the grouping above and is recreated
df['line'] = [re.sub('.+\.', '', line) for line in df['id_line']] #create a line number for sorting
df['line'] = [x.replace('l', '') for x in df['line']]
df

,id_line,label,lemma,id_text,line
0,P235192.10,o 8,istatirru[stater]N ša[of]DET Seleucus[1]RN bab...,P235192,10
1,P235192.11,o 9,ana[to]PRP Kidin-Anu[00]PN māru[son]N ša[of]DE...,P235192,11
2,P235192.12,o 10,ana[to]PRP ūmu[day]N ṣiātu[distant-time]N nadā...,P235192,12
3,P235192.13,o 11,šuāti[him]IP Anu-ah-ittannu[00]PN ina[in]PRP q...,P235192,13
4,P235192.14,o 12,ana[to]PRP muhhu[skull]N mišlu[half]NU ūmu[day...,P235192,14
5,P235192.15,o 13,bašû[exist]V Anu-ah-ittannu[00]PN nādinu[giver...,P235192,15
6,P235192.16,o 14,marāqu[rub]V adi[until]PRP 12-TA.AM₃[NA]NA ana...,P235192,16
7,P235192.17,o 15,nadānu[give]V mišlu[half]NU ūmu[day]N ina[in]P...,P235192,17
8,P235192.19,r 1,ūmu[day]N 29-KAM₂[NA]NA ūmu[day]N 30-KAM₂[NA]N...,P235192,19
9,P235192.20,r 2,Anu-ahhe-iddin[00]PN māru[son]N ša[of]DET Nana...,P235192,20


In [15]:
df['line'] = [int(x) for x in df['line']]
df = df.sort_values(['id_text', 'line']).reset_index(drop=True)
df.head(100)

,id_line,label,lemma,id_text,line
0,P235192.3,o 1,Anu-ah-ittannu[00]PN māru[son]N ša[of]DET Nidi...,P235192,3
1,P235192.4,o 2,ina[in]PRP hūdu[happiness]N libbu[interior]N m...,P235192,4
2,P235192.5,o 3,ūmu[day]N 29-KAM₂[NA]NA ūmu[day]N 30-KAM₂[NA]N...,P235192,5
3,P235192.6,o 4,mahru[front]N Anu[1]DN Antu[1]DN Ištar[1]DN Be...,P235192,6
4,P235192.7,o 5,ša[of]DET arhussu[every-month]AV kalû[totality...,P235192,7
5,P235192.8,o 6,ša[that]REL ana[to]PRP isqu[share]N šuāti[him]...,P235192,8
6,P235192.9,o 7,šuāti[him]IP māru[son]N ša[of]DET Anu-ahhe-idd...,P235192,9
7,P235192.10,o 8,istatirru[stater]N ša[of]DET Seleucus[1]RN bab...,P235192,10
8,P235192.11,o 9,ana[to]PRP Kidin-Anu[00]PN māru[son]N ša[of]DE...,P235192,11
9,P235192.12,o 10,ana[to]PRP ūmu[day]N ṣiātu[distant-time]N nadā...,P235192,12


In [16]:
df = df.loc[np.where(df['lemma'] != '')] # kick out rows with empty lemma field
df

,id_line,label,lemma,id_text,line
0,P235192.3,o 1,Anu-ah-ittannu[00]PN māru[son]N ša[of]DET Nidi...,P235192,3
1,P235192.4,o 2,ina[in]PRP hūdu[happiness]N libbu[interior]N m...,P235192,4
2,P235192.5,o 3,ūmu[day]N 29-KAM₂[NA]NA ūmu[day]N 30-KAM₂[NA]N...,P235192,5
3,P235192.6,o 4,mahru[front]N Anu[1]DN Antu[1]DN Ištar[1]DN Be...,P235192,6
4,P235192.7,o 5,ša[of]DET arhussu[every-month]AV kalû[totality...,P235192,7
5,P235192.8,o 6,ša[that]REL ana[to]PRP isqu[share]N šuāti[him]...,P235192,8
6,P235192.9,o 7,šuāti[him]IP māru[son]N ša[of]DET Anu-ahhe-idd...,P235192,9
7,P235192.10,o 8,istatirru[stater]N ša[of]DET Seleucus[1]RN bab...,P235192,10
8,P235192.11,o 9,ana[to]PRP Kidin-Anu[00]PN māru[son]N ša[of]DE...,P235192,11
9,P235192.12,o 10,ana[to]PRP ūmu[day]N ṣiātu[distant-time]N nadā...,P235192,12


In [17]:
grouped = df['lemma'].groupby(df['id_text']).apply(' '.join).reset_index()
by_text_df = pd.DataFrame(grouped)
by_text_df = by_text_df.set_index('id_text')
by_text_df

,lemma
id_text,
P235192,Anu-ah-ittannu[00]PN māru[son]N ša[of]DET Nidi...
P296411,x[NA]NA kaspu[silver]N erbu[income]N ša[of]DET...
P296639,mimma[anything]XP dibbu[words]N dīnu[legal-dec...
P296677,kuruppu[(vegetable)basket]N ša[of]DET Nanaya-i...
P296678,Anu-ah-ušabši[00]PN māru[son]N ša[of]DET Anu-i...
P296679,Anu-ab-uṣur[00]PN u[and]CNJ Anu-balassu-iqbi[0...
P296680,Labaši[00]PN māru[son]N ša[of]DET Tattannu[00]...
P296681,Ina-buni-Nanaya[00]PN mārtu[daughter]N ša[of]D...
P296682,Ina-buni-Nanaya[00]PN mārtu[daughter]N ša[of]D...


## Save in CSV Format

In [18]:
if name[-4:] == '.txt': # list of P/Q/X numbers in .txt file
    filename = name[:-4]
else:                  # full project
    filename = name.replace('/', '_')
with open('output/' + filename + '.csv', 'w') as w:
    by_text_df.to_csv(w, encoding='utf8')